# G-Research Algorithmic Finance Challenge

### The goal of this challenge was to win as many hands of a Blackjack variant as possible whilst competing against opposing teams. Each team has to submit code written in Python which will then be executed on G-Research's servers along with the submissions of other teams.

This was the setup of the challenge:
 - Rounds are run every 2-4 minutes for 120 minutes
 - Each round 10000 hands are played
 - Player order is shuffled before each hand, however the Dealer always goes last
 - A hand consists of each player taking it in turns to 'hit' or 'stick'
 - Like normal Blackjack (AKA 21), a player can 'hit' as many times as they like, stopping either by 'sticking' or by going bust
 - Unlike normal Blackjack, players start with no cards, will go bust if their cards total 1 or more, and each card has a uniformly random value (0, 1)
 - Also unlike normal Blackjack, the Dealer does not follow a published set of steps, it just follows the same game rules as everyone else and tries to win
 
The scoring went as follows:
 - At the end of each round, teams are ranked based on hand wins
 - The team with the most hand wins gets X points, 2nd place gets X/2 points, 3rd place gets X/3, and so on
 - X is equal to the round number squared, i.e. 10000 in the first round, 40000 in the second, 90000 in the third, and so on.
 - Teams that are tied receive the same number of points, and then a gap is left in the rankings, i.e. if teams A & B tie for first, they both receive X points, and the next team C receives X/3 points
 - 10,000 hand rounds may be split into multiple sub-rounds of 2,500 hands or fewer - each sub-round is scored independently in this case
 - There is no cost to playing a round or hand, and there is no penalty for losing a hand.

For each hand a given team participates in, their Python submission is executed. The method  play_hand() is called (if defined correctly). This method allows for various other methods and variables included in the Challenge API to be used in order to implement the desired playing strategy:
 - hit(): a function you can invoke in order to take a card. It returns the value of your new card. It does not return a cumulative total
 - hand_number: an integer representing the current hand number in this round, indexed from 1
 - player_count: an integer count of the number of players in this round, including the dealer
 - current_hand:  list of named tuples, [(team_id: integer, cards: [float])] containing the cards that each team before you received, delivered as cumulative card totals. If a player didn't hit() at all, then the contents will be [0.0], otherwise it will contain the cumulative totals of their cards, e.g. for a player that hit() three times and went bust, the array would contain [0.0, 0.529, 0.82, 1.5]. The current_hand list will be empty if you are going first in the hand, or will contain player_count - 2 elements if you are playing just before the Dealer.
 team_order: list of integers representing the team ids of each player in this hand, in the order that they will play. This includes the dealer who will always appear last.
 - previous_hands: list of lists, [[(team_id: integer, cards: [float])]] containing the cumulative card totals, as described above, for all teams (including your team), but for all previous hands in this round
 - state: a dictionary that you can use to store cross-hand data. There is no equivalent for cross-round data available.

#### Below are given the submissions of our team throughout the durartion of the challenge

In [ ]:
# very first test

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):
    score = hit()
    if score < 0.5:
        hit()

This is the very first submission, used as test to get the hang of the software. We suspected that, since the expected value of a number chosen at random from a uniform distribution is 0.5, we should only hit if our current sum is below 0.5.

In [ ]:
# version 0

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):
    score = 0
    while score < 0.5:
        score += hit()

For our second submission, we noted that we either win or lose in this game, i.e., even if we don't go bust, we still lose if we don't have the highest sum at the end of the hand. Hence, we decided to hit if our current sum is below the highest winning, i.e., below 1, sum achieved by any of the teams before us in the hand. If we currently have the winning sum, we hit while our sum is below 0.5.

In [ ]:
# version 1

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):
    
    # Find the winning score achieved by a team before us in the current hand:
    best_score_so_far = 0
    for team_info in current_hand:
        team_score = team_info.cards[-1]
        if team_score < 1 and team_score > best_score_so_far:
            best_score_so_far = team_score
    
    score = 0
    while score < 1:
        score += hit()
        if score > 0.5 and score > best_score_so_far:
            return

For the next version, we decided that a good strategy would be to play more aggressively depending on how early our turn is in the current hand. Initially, we just set the goal for our sum (which was 0.5 in the previous two versions) to 0.9 if we were the first team to draw.

In [ ]:
# version 2 - aggressive play

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):
    
    # Find the winning score achieved by a team before us in the current hand:
    best_score_so_far = 0
    for team_info in current_hand:
        team_score = team_info.cards[-1]
        if team_score < 1 and team_score > best_score_so_far:
            best_score_so_far = team_score
    
    # Determine how many teams will draw before us in the current hand:
    count_before = len(current_hand)
    count_after = player_count - count_before - 1
    
    if count_before == 0:
        goal = 0.9 # Set the aggressive goal if we are playing first in the hand
    else:
        goal = 0.5
    
    score = 0
    while score < 1:
        
        card = hit()
        score += card
        
        if score > goal and score > best_score_so_far:
            return

We then improved the aggressive strategy by modifying the goal based on when exactly we are  going to draw in the current hand. If we were the first team to draw, we set the goal as 0.88, whereas if we were the last (right before the dealer), we set it at the usual 0.5. If we are somewhere in the middle of the order, we just vary the goal linearly from 0.88 to 0.5.

In [ ]:
# version 2.1 - aggressive play, modified

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):
    
    # Find the winning score achieved by a team before us in the current hand:
    best_score_so_far = 0
    for team_info in current_hand:
        team_score = team_info.cards[-1]
        if team_score < 1 and team_score > best_score_so_far:
            best_score_so_far = team_score
    
    # Determine how many teams will draw before us in the current hand:
    count_before = len(current_hand)
    count_after = player_count - count_before - 1
    
    # Set the goal based on our place in the current hand:
    goal = 0.88 - ((0.88-0.5)/player_count) * count_before
    
    score = 0
    while score < 1:
        card = hit()
        score += card
        
        if score > goal and score > best_score_so_far:
            return

For the next version, we decided a good strategy would be to determine our goal based on what the average winning sum was for the previous hands played in this round.

In [1]:
# version 3 - dynamically change goal

def play_hand(hit, hand_number, player_count, current_hand, team_order, previous_hands, state):

    # Check if we are in the first hand played for the current round, and initialise the 
    # following entries in the cross-hand dictionary.
    if len(previous_hands) == 1:
        state['cumulative_win_score'] = 0
        state['average_score'] = 0
        print(previous_hands)
        
    # Analyse the previous hand if we are not in the first hand:
    if len(previous_hands) > 1:
        previous_hand = previous_hands[-1] # most recent hand played before the current
        print(previous_hand)
        previous_hand_winning_score = 0

        # Determine the winning sum for this previous hand:
        for team_info in previous_hand:
            team_score = team_info.cards[-1]
            if team_score < 1 and team_score > previous_hand_winning_score:
                previous_hand_winning_score = team_score
        
        # Add this winning sum to the total of the winning sums across all previous hands:
        state['cumulative_win_score'] += previous_hand_winning_score
        # Calculate the new average winning sum:
        state['average_score'] = state['cumulative_win_score']/len(previous_hands)
    
    print('Average_score: ', state['average_score'])
    
    # Find the winning score achieved by a team before us in the current hand:
    best_score_so_far = 0
    for team_info in current_hand:
        team_score = team_info.cards[-1]
        if team_score < 1 and team_score > best_score_so_far:
            best_score_so_far = team_score
    
    # Determine how many teams will draw before us in the current hand:
    count_before = len(current_hand)
    count_after = player_count - count_before - 1
    
    # Set our goal for the sum:
    if len(previous_hands) < 10:

        # If we are playing one of the first 10 hands, set the goal to 0.87 since the average
        # winning sum is not representative enough yet:
        goal = 0.87
    else:
        # Otherwise, set the goal to the average winning sum, or lower depending on what place
        # we draw in for this current hand:
        goal = state['average_score'] - ((state['average_score']-0.5)/player_count) * count_before
    
    score = 0
    while score < 1:
        card = hit()
        score += card
        
        if score > goal and score > best_score_so_far:
            return

### Using the code above, our team was able to finish 2nd in the challenge out of roughly 25 teams. We played a total of 450,000 hands, and won 3.97% of them.